In [2]:
import pytest
import ipytest
ipytest.autoconfig()

function1 ---> connect to buckect and retrieve the objects from the bucket


In [3]:
import boto3
import pyarrow.parquet as pq
import pandas as pd
bucket = 'rand1'
s3_client = boto3.client("s3")
with open("./load_test_db/hello_test.parquet", "rb") as f:
    s3_client.upload_fileobj(f, f"{bucket}", "hello.parquet")
bucket_objects = s3_client.list_objects_v2(Bucket = bucket)
assert bucket_objects['KeyCount'] == 1
file_paths = [f's3://{bucket}/{obj["Key"]}' for obj in bucket_objects['Contents']]
print(file_paths)
for file_path in file_paths:
    s3_file = pq.ParquetDataset(file_path)
    table = s3_file.read().to_pandas()
    df= pd.DataFrame(table)
    print(df)
assert df.columns.tolist() == ['hello']

['s3://rand1/hello.parquet']
   hello
0  world


In [4]:
import boto3
def retrive_files(bucket):
    s3_client = boto3.client("s3")
    bucket = s3_client.list_objects_v2(Bucket = bucket)
    return bucket


In [13]:
%%ipytest
from moto import mock_s3
import pytest
import pyarrow.parquet as pq
import pg8000
from pyarrow import fs

@pytest.fixture(scope='function')
def aws_credentials():
    """Mocked AWS Credentials for moto."""
    os.environ['AWS_ACCESS_KEY_ID'] = 'test'
    os.environ['AWS_SECRET_ACCESS_KEY'] = 'test'
    os.environ['AWS_SECURITY_TOKEN'] = 'test'
    os.environ['AWS_SESSION_TOKEN'] = 'test'
    os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

@mock_s3
def test_creating_mock_s3():
    conn = boto3.client("s3", region_name="us-east-1")
    res = conn.create_bucket(Bucket="test_bucket_29")
    assert res['ResponseMetadata']['HTTPStatusCode'] == 200

@mock_s3
def test_accessing_the_objects_in_the_bucket():
    conn = boto3.client("s3", region_name="us-east-1")
    res = conn.create_bucket(Bucket="test_bucket_29")
    parquet_files = retrive_files('test_bucket_29')
    assert parquet_files['KeyCount'] == 0

@mock_s3
def test_retreiving_objects_in_the_bucket():
    bucket = "test_bucket_29"
    conn = boto3.client("s3")
    res = conn.create_bucket(Bucket=bucket)
    parquet_files = retrive_files(bucket)
    # uploading some fake data to bucket
    with open("./load_test_db/hello_test.parquet", "rb") as f:
        conn.upload_fileobj(f, f"{bucket}", "hello.parquet")
    parquet_files = retrive_files(bucket)
    assert parquet_files['KeyCount'] == 1
    
@pytest.fixture
def test_retreiving_objects_in_the_bucketss():
    bucket ="rand2"
    s3_client = boto3.client("s3")
    res = s3_client.create_bucket(Bucket=bucket,CreateBucketConfiguration={'LocationConstraint': 'eu-west-2'})
    with open("./load_test_db/hello_test.parquet", "rb") as f:
        s3_client.upload_fileobj(f, f"{bucket}", "hello.parquet")
    bucket_objects = retrive_files(bucket)
    file_paths = [f's3://{bucket}/{obj["Key"]}' for obj in bucket_objects['Contents']]
    for file_path in file_paths:
        s3_file = pq.ParquetDataset(file_path)
        table = s3_file.read().to_pandas()
        df= pd.DataFrame(table)
    assert df.columns.tolist() == ['hello']

# @pytest.fixture
# def test_warehouse_connection():
#     connect = make_warehouse_connection()
#     assert isinstance(connect, pg8000.Connection)  


...                                                                                          [100%]
3 passed in 1.25s


In [6]:
import os
from dotenv import load_dotenv
from os.path import join, dirname
from pathlib import Path
import pg8000
def make_warehouse_connection():
    # dotenv_path = join(dirname(__file__), '../config/.env.data_warehouse')
    dotenv_path = Path('./config/.env.data_warehouse')
    load_dotenv(dotenv_path)
    API_HOST =  os.environ["host"]
    API_USER = os.environ["user"]
    API_PASS = os.environ["password"]
    API_DBASE = os.environ["database"]
    conn = pg8000.connect(
        host=API_HOST,
        user=API_USER,
        password=API_PASS,
        database=API_DBASE
    )
    return conn

print(make_warehouse_connection())


In [21]:
import io
import pyarrow.parquet as pq
import pandas as pd
import pg8000
from pathlib import Path
import os
from dotenv import load_dotenv


def get_data(bucket_name, file_path):
    s3 = boto3.client('s3')
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=file_path)['Contents']
    dfs = {}
    for obj in objects:
        key = obj['Key']
        filename = key.split('/')[-1].split('.')[0]
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        buffer = io.BytesIO(obj['Body'].read())
        table = pq.read_table(buffer)
        df = table.to_pandas()
        dfs[f"df_{filename}"] = df
    return dfs


def make_warehouse_connection(dotenv_path="./config/.env.data_warehouse"):
    try:
        dotenv = Path(dotenv_path)
        load_dotenv(dotenv)
        API_HOST = os.environ["host"]
        API_USER = os.environ["user"]
        API_PASS = os.environ["password"]
        API_DBASE = os.environ["database"]
        conn = pg8000.connect(
            host=API_HOST,
            user=API_USER,
            password=API_PASS,
            database=API_DBASE
        )
        return conn
    except Exception as e:
        print(f"Error connecting to the data warehouse: {str(e)}")
        return None



In [8]:
import boto3
bucket_name = 'rand6'
file_path = 'data/parquet'
dotenv_path = './config/.env.test'
s3_client= boto3.resource('s3')
res = s3_client.create_bucket(Bucket=bucket_name)
s3_client.Object(bucket_name, f"{file_path}/dim_currency.parquet").put(
        Body=open('./load_test_db/dim_currency.parquet', 'rb'))
s3_client.Object(bucket_name, f"{file_path}/dim_date.parquet").put(
        Body=open('./load_test_db/dim_date.parquet', 'rb'))
s3_client.Object(bucket_name, f"{file_path}/hello.parquet").put(
        Body=open('./load_test_db/hello_test.parquet', 'rb'))        


{'ResponseMetadata': {'RequestId': '7B3669CW6W3WVNZS',
  'HostId': 'qnp9SVonGE9TTVzobTtAR7voAjckgowSm9WQvI5d/pisFkNuxt8wVqv9G2c6raqyLY0CZrb3FKM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qnp9SVonGE9TTVzobTtAR7voAjckgowSm9WQvI5d/pisFkNuxt8wVqv9G2c6raqyLY0CZrb3FKM=',
   'x-amz-request-id': '7B3669CW6W3WVNZS',
   'date': 'Fri, 31 Mar 2023 08:51:28 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4f231988a0ecfb93deda19f91db2bf81"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4f231988a0ecfb93deda19f91db2bf81"',
 'ServerSideEncryption': 'AES256'}

In [41]:
df = get_data(bucket_name,file_path)
conn = make_warehouse_connection(dotenv_path)

In [42]:
cur = conn.cursor()
for table in df:
    table_name = table[3:]
    print(f"Loading table {table_name}")
    for index, row in df[table].iterrows():
        values = ', '.join(['%s'] * len(row))
        columns = ', '.join(row.index)
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        cur.execute(sql, tuple(row))
conn.commit()
cur.close()
conn.close()

Loading table dim_currency
Loading table dim_date


In [2]:
import json
with open('./tests/test_data/incorrect_bucket.json') as i:
    json=json.loads(i.read())
print (json)

{'Records': [{'eventVersion': '2.0', 'eventSource': 'aws:s3', 'awsRegion': 'us-east-1', 'eventTime': '1970-01-01T00:00:00.000Z', 'eventName': 'ObjectCreated:Put', 'userIdentity': {'principalId': 'EXAMPLE'}, 'requestParameters': {'sourceIPAddress': '127.0.0.1'}, 'responseElements': {'x-amz-request-id': 'EXAMPLE123456789', 'x-amz-id-2': 'EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH'}, 's3': {'s3SchemaVersion': '1.0', 'configurationId': 'testConfigRule', 'bucket': {'name': 'wrong_bucket', 'ownerIdentity': {'principalId': 'EXAMPLE'}, 'arn': 'arn:aws:s3:::example-bucket'}, 'object': {'key': 'data/parquet/dim_date.parquet', 'size': 1024, 'eTag': '0123456789abcdef0123456789abcdef', 'sequencer': '0A1B2C3D4E5F678901'}}}]}
